In [1]:
import getpass
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_chroma import Chroma
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlsplit, urlunsplit

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = init_chat_model("gpt-4o-mini", model_provider="openai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  
)

In [22]:
def normalize_url(url):
    parts = urlsplit(url)
    path = parts.path.rstrip("/") or "/"
    return urlunsplit((parts.scheme, parts.netloc, path, "", ""))

def is_page_url(url):
    forbidden_schemes = ['mailto', 'tel', 'ftp', 'file', 'webcal', 'ws', 'wss']
    file_exts = [
        ".pdf", ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".svg", ".doc", ".docx",
        ".xls", ".xlsx", ".ppt", ".pptx", ".zip", ".rar", ".mp3", ".mp4", ".avi", ".mov",
        ".txt", ".csv", ".xml", ".json", ".xlsx", ".tar", ".gz", ".7z", ".ico"
    ]
    parsed = urlparse(url)
    if parsed.scheme not in ('http', 'https'):
        return False
    if parsed.scheme in forbidden_schemes:
        return False
    path = parsed.path.split("?")[0]
    _, ext = os.path.splitext(path)
    if ext.lower() in file_exts:
        return False
    return True

def get_internal_links(base_url):
    try:
        r = requests.get(base_url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
    except Exception as e:
        return []
    base_domain = urlparse(base_url).netloc.replace("www.", "")
    urls = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if not href or href.startswith(("#", "mailto:", "tel:")):
            continue
        url = urljoin(base_url, href)
        link_domain = urlparse(url).netloc.replace("www.", "")
        if link_domain != base_domain:
            continue
        norm_url = normalize_url(url)
        if is_page_url(norm_url):
            urls.add(norm_url)
    return list(urls)


def crawl_site(start_url, max_depth=2):
    visited = set()
    to_visit = [(start_url, 0)]
    all_internal = set()

    while to_visit:
        url, depth = to_visit.pop()
        if url in visited or depth > max_depth:
            continue
        visited.add(url)
        links = get_internal_links(url)
        all_internal.update(links)
        if depth < max_depth:
            for link in links:
                if link not in visited:
                    to_visit.append((link, depth + 1))
    return list(all_internal)


In [23]:
urls = crawl_site("https://www.unne.edu.ar/")

In [24]:
print(urls)

['https://www.unne.edu.ar/agenda/eventos/sesion-de-consejo-superior-2025-05-07', 'https://www.unne.edu.ar/agenda/eventos/introduccion-a-las-microcredenciales-academicas-y-su-relacion-con-la-ead', 'https://www.unne.edu.ar/agenda/eventos/capacitacion-inteligencia-artificial-para-emprendedores', 'https://www.unne.edu.ar/estudiar/posgrado/doctorados', 'https://www.unne.edu.ar/agenda/eventos/my-bookings', 'https://www.unne.edu.ar/estudiar/inscripciones', 'https://www.unne.edu.ar/obras/ampliacion-de-plazos-refaccion-del-sector-de-higiene-y-seguridad-de-comunicacion-institucional-y-dpto-de-higiene-y-seguridad', 'https://www.unne.edu.ar/concursos/facultad-de-ciencias-economicas-mejoramiento-de-las-dedicaciones-docente-res-no-1117-24-d', 'https://www.unne.edu.ar/nosotros/comunicacion-e-imagen-institucional', 'https://www.unne.edu.ar/agenda/eventos/lanzamiento-concurso-emprendimiento-argentino-2025', 'https://www.unne.edu.ar/agenda/eventos/viii-jornadas-de-auditores-de-universidades-nacionales/i

In [33]:
print(len(urls))

596


In [25]:
# Load and chunk contents of the blog
loader = WebBaseLoader(web_paths=urls)
docs = loader.load()

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

MAX_BATCH = 5461

# Index en lotes
for batch in chunks(all_splits, MAX_BATCH):
    vector_store.add_documents(documents=batch)

In [31]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

c:\Users\johan\Documents\code\ia_ps\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [41]:
response = graph.invoke({"question": "Entre los años 1920 y 1955 que ocurrio en la UNNE"})
print(response["answer"])

Entre 1920 y 1955, la Universidad Nacional del Nordeste (UNNE) se enfocó en la expansión territorial y en democratizar el acceso a la educación superior, estableciendo dependencias académicas y administrativas fuera de sus sedes centrales. Esto permitió a comunidades distantes de los centros urbanos acceder a la educación. La UNNE sentó las bases para su actual estructura, que incluye varios Centros Regionales Universitarios.


In [42]:
response = graph.invoke({"question": "SCI-DI UNNE: nueva convocatoria a Categorización Interna de D-I de la UNNE"})
print(response["answer"])

La Secretaría General de Ciencia y Técnica de la UNNE ha lanzado una nueva convocatoria para la Categorización Interna de Docentes Investigadores, abierta desde el 5 de mayo hasta el 27 de mayo de 2025. Los docentes podrán postular a través del sistema SAP y solicitar una categoría entre V e I, adjuntando la documentación necesaria. Esta convocatoria se enmarca en el reglamento aprobado por la Resolución Nº 2305/25 R.


In [43]:
response = graph.invoke({"question": "Requisitos para el ingreso de mayores SIN Título Secundario:"})
print(response["answer"])

Los requisitos para el ingreso de mayores de 25 años sin título secundario incluyen la presentación de una fotocopia del documento de identidad, constancia de estudios primarios completos, y certificados que acrediten experiencia laboral relacionada con la carrera elegida. Además, es necesario presentar constancia del último ciclo de enseñanza secundaria que indique las asignaturas adeudadas y una copia del carnet de vacunación correspondiente. Todos los documentos enviados tendrán carácter de declaración jurada y estarán sujetos a verificación por la unidad académica.
